# Empowerment: Pacman

In [79]:
import numpy as np
from itertools import combinations_with_replacement, combinations, permutations, product
import pygame

background_color = [0, 0, 0]
pacman_color = [255, 255, 0]
wall_color = [255, 255, 255]
ghost_color = [0, 0, 255]
blocksize = 26
agentsize = 10
ghostsize = 20

In [80]:
# ' ' - nothing
# 'P' - player
# '#' - wall
# 'G' - ghost

field = np.array(
        [[' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', '#', '#', '#', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', '#', ' ', '#', '#', '#', '#', ' ', ' ', ' ', '#', 'G', '#', ' '],
         ['#', '#', '#', 'P', '#', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', '#', '#', '#', '#', ' ', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', 'G', ' ', ' ', ' ', ' ', '#', ' ', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', '#', ' '],
         [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', '#', '#', '#', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', '#', '#', '#', '#', ' ', ' ', ' ', '#', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
         [' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'G', ' ']])

height = len(field)
width = len(field[0])

In [81]:
steps = 4

def update(current):
    return current

def actions_P(field):
    # position of variable (index)
    Px, Py = np.where(field == 'P')
    # field[int(x)][int(y)] 
    
    # number of possible actions: 4
    #    -0-
    #    3P1
    #    -2-
    # calculation of possible steps from future position (0 to 3)
    paths = set()
    for i in product(range(steps), repeat=steps):
        paths.add(i)
    
    # iterate over field around P
    # 1. calculate free postions around you, 2. calculate number of actions possible from free positions
    counts = []

    
    # for i in range(height):
        # for j in range(width):
            
    return

In [82]:
paths = set()
for i in product(range(4), repeat=4):
    paths.add(i)

In [83]:
# Returns the resulting position of a moving object, given its current position and its action
def update_position(position, action):
    newpos = [position[0] + action[0], position[1] + action[1]]
    newpos[0] %= height
    newpos[1] %= width
    return newpos

In [84]:
def player_position(field):
    return np.array(np.where(field == 'P')).flatten()

In [85]:
# Returns a list of possible actions (lists with relative y and x positions), given the current field and angent position
def possible_actions(field, position):
    actions = []
    for action in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
        newpos = update_position(position, action)
        target = field[newpos[0]][newpos[1]]
        if target == '#':
            continue
        elif target == 'G':
            continue
        actions.append(action)
    return actions

In [86]:
# Updates a field matrix and moves an agent from the old position to a new position
def move_agent(field, old_position, new_position):
    agent = field[old_position[0]][old_position[1]]
    target = field[new_position[0]][new_position[1]]
    if not (agent == 'P' and target == 'G'):
        field[new_position[0]][new_position[1]] = agent
    field[old_position[0]][old_position[1]] = ' '
    return field

#print(move_agent(field, [5, 3], [3, 3]))

In [87]:
def update(field, player_action):
    def manhattan(p_x, p_y, g_x, g_y):
        return abs(p_x - g_x) + abs(p_y - g_y)
    
    ## calculation of ghost actions 
    # ghost positions
    ghost_x = []
    ghost_y = []
    ghost_x, ghost_y = np.where(field == 'G')
    
    player_pos = np.array(np.where(field == 'P')).flatten()

    for ghost_pos in zip(ghost_x, ghost_y):
        actions_ghost = possible_actions(field, ghost_pos)
        distance = 100 # high value
        for action in actions_ghost:
            possible_pos = update_position(ghost_pos, action)
            curr_distance = manhattan(player_pos[0], player_pos[1], possible_pos[0], possible_pos[1])
            
            if curr_distance < distance: 
                distance = curr_distance
                best_pos = possible_pos
        field = move_agent(field, ghost_pos, best_pos)
    new_player_pos = update_position(player_pos, player_action)
    return move_agent(field, player_pos, new_player_pos)

In [88]:
# Returns an array of fields which result in one of the possible player actions being taken
# (starting from every field in the input array)
def possible_fields(fields):
    possibilities = []
    for field in fields:
        for action in possible_actions(field, player_position(field)):
            possibilities.append(update(field, action))
    return possibilities

In [89]:
# Returns an empowerment measure calculated from an array of reachable fields
def empowerment(fields):
    return np.log(len(set([tuple(tuple(element) for element in field) for field in fields])))
    
#print(empowerment(possible_fields([field, field])))

In [90]:
pygame.init()
clock = pygame.time.Clock()

size = [width * blocksize, height * blocksize]
screen = pygame.display.set_mode(size)
pygame.display.set_caption("Pacman")

done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
 
    screen.fill(background_color)
    for x in range(0, width):
        for y in range(0, height):
            if(field[y][x] == '#'):
                pygame.draw.rect(screen, wall_color, [x * blocksize, y * blocksize, blocksize - 1, blocksize - 1])
            elif(field[y][x] == 'P'):
                pygame.draw.circle(screen, pacman_color, [int(x * blocksize + blocksize / 2), int(y * blocksize + blocksize / 2)], agentsize)
            elif(field[y][x] == 'G'):
                pygame.draw.rect(screen, ghost_color, [int(x * blocksize + (blocksize - ghostsize) / 2),
                                                       int(y * blocksize + (blocksize - ghostsize) / 2),
                                                       ghostsize - 1, ghostsize - 1])            
    pygame.display.flip()
    clock.tick(20)
    
pygame.quit()